In [1]:
!pip install CFEDemands --upgrade
!pip install xarray
# !pip install dvc
#!pip install dvc --ignore-installed
!pip install --upgrade oauth2client
!pip install -r requirements.txt

Requirement already up-to-date: CFEDemands in /home/dan/anaconda3/lib/python3.8/site-packages (0.2.7)
Requirement already up-to-date: oauth2client in /home/dan/anaconda3/lib/python3.8/site-packages (4.1.3)


In [2]:
import pandas as pd

# set dataframes to show float values without exponential
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## (#A) Estimate Demand System

Estimate a system of demands for different kinds of food. Characterize how consumption varies with household need.

Retrieve data from google sheets using private keys:

In [ ]:
import pandas as pd
import numpy as np
from eep153_tools import read_sheets

#Nigeria consumption (own production) dataset
Nigeria_Data = '17L5cDhXRLNAckP3JvBLTLSYIguFqP2ebMvQLH96c0n4'

#### Need private keys from json file (we're authenticating using "service accounts")
!gpg --batch --passphrase "noodle octopus" -d ./students-9093fa174318.json.gpg > ./students-9093fa174318.json
####

# Add credentials if sheet not meant to be public
exp_df = read_sheets(Nigeria_Data,sheet='Expenditures',json_creds='./students-9093fa174318.json')
                 
# Change 'ICRISAT' to key of your own sheet in Sheets, above
household_df = read_sheets(Nigeria_Data,sheet="HH Characteristics",json_creds='./students-9093fa174318.json')

# # Assume a single market by setting m = 1 for all
exp_df.insert(loc=2, column='m', value=1)
household_df.insert(loc=2, column='m', value=1)

household_df2 = household_df.copy().set_index(['t', 'j', 'm'])

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


### Data Cleaning and filter data by our chosen population (below poverty line)

In [ ]:
exp_df = exp_df.replace(0,np.nan) # Replace zeroes with np.nan

#Convert all value types to float64
for i in range(3, len(exp_df.columns)):
    exp_df.iloc[:, i:] = exp_df.iloc[:, i:].astype('float64')
    
exp_df = exp_df.replace(0.0,np.nan) # Replace zeroes with np.nan

#### Create total expenditure column and filter expenditure dataframe by our condition

We create a new column 'Total expenditure' so we can filter households that have total food spendings under or at poverty line.

In [ ]:
exp_df['Total expenditure'] = exp_df.iloc[:, 3:].sum(axis=1)
pd.DataFrame(exp_df['Total expenditure'])

#### Poverty Line Dataframe (Households with food spendings under or equal to poverty line)

In [ ]:
#87.8 thousand Naira food spending per year
#1688.46 per week
wk_poverty_line = 87800/52

poverty_line_df = exp_df[exp_df['Total expenditure'] <= wk_poverty_line]
poverty_line_df

In [ ]:
#remove total expenditure column from calculation
poverty_line_df = poverty_line_df.drop(columns=['Total expenditure']) 
# poverty_line_df.set_index(['j', 't', 'm'])
poverty_line_df

#### Filter household_df to include those only at or below poverty_line:

In [ ]:
# filter households to those only at or below poverty line
idx_selected_households = poverty_line_df.index.tolist()
household_df = household_df[household_df.index.isin(idx_selected_households)]
household_df

#### Logged Food Expenditure Dataframe (after running np.log on values)

In [ ]:
# Take logs of expenditures of our selected population and name it log_poverty_line_df
log_poverty_line_df = np.log(poverty_line_df.set_index(['j', 't', 'm']))
log_poverty_line_df

#### Add 'log Hsize' column to household_df:

In [ ]:
# set index to j, t, m so that df.sum() ignore index values
household_df.set_index(['j','t','m'], inplace=True)

# create new column of household size
household_df['Hsize'] = household_df.sum(axis=1).values

# remove erroneous data with household_size = 0
household_df = household_df[household_df['Hsize'] > 0]

# create new column 'log Hsize'
household_df['log Hsize'] = np.log(household_df['Hsize'])

# remove Hsize column
household_df = household_df.drop(columns=['Hsize']) 

#### Household Characteristic Table (Only includes households at/below poverty line)

In [ ]:
household_df

### Estimation



With nothing more than this, we can estimate the demand system.  This
happens in two steps.  The first is the &ldquo;reduced form&rdquo; step:



In [ ]:
import cfe

#regressand y is food expenditures (np.logged) by our selected population
#regressor z is household demographics

result = cfe.Result(y=log_poverty_line_df,z=household_df, min_xproducts=15, min_proportion_items=0.05)

This creates a complicated &ldquo;Result&rdquo; object, with lots of different
attributes.  Note from below that attributes $y$ and $z$ are now defined.



In [ ]:
result

In [ ]:
# result.get_loglambdas(as_df=True).quantile(0.8)

### First step of Estimation



Recall that there are two steps to estimation; the first step
involves estimating the &ldquo;reduced form&rdquo; linear regression 
$$
y_{it}^j = {a}_{it} + \delta_i'{z}^j_t + \epsilon_{it}^j.
$$

The Result class has code to estimate this in one line:



In [ ]:
result.get_reduced_form()

After running this we can examine the estimated coefficients $\delta$:



In [ ]:
result.delta.to_dataframe().unstack('k')

Also the good-time constants $a_{it}$ (this captures the effects of prices)<br>
i.e. differences in year and alpha coefficient


In [ ]:
result.a.to_dataframe().unstack('i')

### Second step of Estimation



The second step involves using Singular Value Decomposition to find
the rank one matrix that best approximates the residuals $e_{it}^j$.
This can be interpreted as
$$
    -\beta_i\log\lambda^j_t,
$$
where the $\log\lambda^j_t$ is the log of the marginal utility of
expenditures (MUE) for household $j$ at time $t$, and where $\beta_i$ are
the corresponding &ldquo;Frisch elasticities&rdquo; that tell us how much
demand changes as the MUE falls.

Estimates can also be computed as a one-liner:



In [ ]:
result.get_beta(as_df=True)

That&rsquo;s all there is to estimation!  Note that we didn&rsquo;t estimate
demands for all goods&#x2014;lots of goods didn&rsquo;t have enough observations,
and were automatically dropped.  (This can be controlled using the
`min_proportion_items` and `min_xproducts` attributes when one
instantiates the result object.)



### Assessment of Fit



Now, let&rsquo;s see how we did, by comparing total expenditures predicted by the
model we&rsquo;ve estimated with actual total expenditures:



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm

xbar = np.exp(result.y).sum(['m','i']).to_dataframe('xbar').replace(0,np.nan).squeeze()
xhat = result.get_predicted_expenditures().sum(['m','i']).to_dataframe('xhat').replace(0,np.nan).squeeze()

# Make dataframe of actual & predicted
df = pd.DataFrame({'Actual':np.log(xbar),'Predicted':np.log(xhat)})

df.plot.scatter(x='Predicted',y='Actual')

# Add 45 degree line
v = plt.axis()
vmin = np.max([v[0],v[2]])
vmax = np.max([v[1],v[3]])
plt.plot([vmin,vmax],[vmin,vmax])

## (B) Nutritional content of different foods

For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances.

#### Load in Nigeria Consumption dataframe:

In [ ]:
import pandas as pd
import numpy as np
from eep153_tools import read_sheets

url = '1kG_fVBmj9EEF9LOwxN30HBxkQENOoWeQjVPYzMJe3b4'

#### Need private keys from json file (we're authenticating using "service accounts")
!gpg --batch --passphrase "noodle octopus" -d ./students-9093fa174318.json.gpg > ./students-9093fa174318.json
####

# Add credentials if sheet not meant to be public
consumption_df = read_sheets(key=url ,sheet='Consumption',json_creds='./students-9093fa174318.json')

# Consider consumption of only those in our selected households
consumption_df.insert(loc=2, column='m', value=1)

In [ ]:
selected_household_ids = exp_df.index.tolist()
selected_household_ids

In [ ]:
# remove erraneous column 'Canned'
consumption_df = consumption_df.drop(columns=['Canned'])

consumption_df = consumption_df[consumption_df.index.isin(selected_household_ids)]
consumption_df

#### Loading in fdc_codes from google sheets:

In [ ]:
# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --batch --passphrase "casimir" -d project3-key.json.gpg > project3-key.json

serviceacct = {'project2-casimir-funk':'project3-key.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1O0nHroRtnzBKSZXVWlqdTLIX16n5gg8aVBPcTjvLFKo/edit#gid=0"
sheet_name = 'Codes'

# Add credentials if sheet not meant to be public
fdc_code_df = read_sheets(key=sheet_url, json_creds=serviceacct["project2-casimir-funk"], sheet=sheet_name)
fdc_code_df

In [ ]:
#list of foods consumed:
fdc_food_items = fdc_code_df['Food description'].tolist()
fdc_food_items

In [ ]:
#example: accessing fdc code from the dataframe
f = fdc_food_items[29]
fdc_id = int(fdc_code_df[fdc_code_df['Food description'] == f]['USDA FDC ID'].values[0])
fdc_id

In [ ]:
import fooddatacentral as fdc

# API key for FDC
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"


# Query to fdc and get dataframe containing nutritional data of foods using their fdc_ids
food_to_nutrients = {}
for f in fdc_food_items:
    fdc_id = fdc_code_df[fdc_code_df['Food description'] == f]['USDA FDC ID'].values[0]
    if not np.isnan(fdc_id):
        try:
            fdc_id = int(fdc_id)
            food_to_nutrients[f] = fdc.nutrients(apikey, fdc_id).Quantity
        except AttributeError:
            print(fdc_id)
            pass
#             warnings.warn("Couldn't find FDC Code %s for food %s." % (f, fdc_id))

nutritional_df = pd.DataFrame(food_to_nutrients,dtype=float).fillna(0.0)
nutritional_df

## (B) Nutritional adequacy of diet

Given the food actually consumed in your data, what can you say about the adequacy of the diets in the population you're studying? What proportion of households consume enough so that members will exceed dietary recommendations? What proportion do not?

#### Dietary Reference Intakes Dataframe:

In [ ]:
dri_df = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
dri_df

### Data Cleaning: Convert consumption dataframe values to hectograms

#### Get all unit types in the consumption dataframe:

In [ ]:
# household_idx = index list of households
consumption_df = consumption_df.set_index(['t', 'j', 'm'])
household_idx = consumption_df.index

In [ ]:
# returns consumption_df of a household, when given household index

def get_h_consumption_df(idx):
    # access consumption_df of a particular household
    h_con = consumption_df.loc[idx]
    h_con = h_con.fillna(0)
    
    #set index to food unit
    h_con = h_con.set_index('u')
    
    #remove 0 values i.e. food not eaten by household
    h_con = h_con.loc[:, h_con.sum(axis=0) != 0.0]
    
    return h_con

In [ ]:
# returns unit type of a food item

def get_unit(df, food):
    unit_idx = df[food].to_numpy().nonzero()[0][0]
    return df.index[unit_idx]

In [ ]:
def add_units_to_set(h_df, unitset):
    for col in h_df.columns:
        unit = get_unit(h_df, col)
        unitset.add(unit)

unitset = set()

for i in household_idx:
    h_df = get_h_consumption_df(i)
    add_units_to_set(h_df, unitset)

unitset

#### Create a dictionary which maps unit type to multiplier value in unit to hectogram conversion:
i.e. dict['Kilograms'] = 10, since 1KG = 10 Hectogram

In [ ]:
keys = list(unitset)
values = [150, 200, 400, 30, 10, 50, 10, 500, 100, 80, 300, 0.1, 10, 10, 50, 150, 600, 80, 0.01, 0.01, 500, 250, 0.01]
unit_map_dict = dict(zip(keys, values))
unit_map_dict

#### Convert consumption dataframe units to hectograms (hectograms_consumption_df):

In [ ]:
hect_consumption_df = consumption_df.set_index('u', append=True)

for i in hect_consumption_df.index:
    k = unit_map_dict[i[3]] 
    hect_consumption_df.loc[i] *= k

hect_consumption_df = hect_consumption_df.reset_index()
hect_consumption_df = hect_consumption_df.set_index(['t', 'j', 'm'])

# Rename all unit values to Hectogram
hect_consumption_df['u'] = hect_consumption_df['u'].apply(lambda x: 'Hectograms')
hect_consumption_df
    
hect_consumption_df

In [ ]:
# function that returns household consumption when given df, and household idx

def get_household_hecto_df(df, h_idx):
    df = hect_consumption_df.loc[idx]
    df = df.groupby(by=df.index, as_index=True).sum()
    return df

In [ ]:
# calls function to return consumption dataframe of household
get_household_hecto_df(hect_consumption_df, household_idx[0])

### Household adequacy of diet calculations

#### Function to get the nutritional content of a certain food:

In [ ]:
def get_nutri(food_name):
    return nutritional_df[food_name]

#### Function to get nutritional values based on total consumption of a particular household:

In [ ]:
# takes in household index, and the list of nutritions to display

def get_total_nutri_household_df(idx, nutritions):
    h_hecto_df = get_household_hecto_df(hect_consumption_df, idx)
    total = 0
    for col in h_hecto_df.columns:
        amount = h_hecto_df[col][0]
        nutri = amount * get_nutri(col)
        total += nutri
    total = total[total.index.isin(nutritions)]
    
    # convert weekly data values to daily values
    total = total / 7
    total = total.rename('Total Nutritional Data for Household: ' + str(idx))
    return pd.DataFrame(total)

In [ ]:
# we are only concern with nutritions in dietary reference intakes
nutritions = dri_df.index

get_total_nutri_household_df(household_idx[0], nutritions)

#### Function that returns household characteristics:

In [ ]:
def get_household_char(idx):
    return household_df2.loc[idx]

#### Function that returns total dietary requirement of a selected household:

In [ ]:
def get_dri_household_df(idx, dri_df):
    # household characteristics
    hc = household_df2.loc[idx]
    
    # loop through every age group and sum up their dri requirements
    total = 0
    for age_g in hc.index:
        if age_g == 'M 0-3' or age_g == 'F 0-3':
            total += dri_df['C 1-3'] * hc[age_g]
        else:
            total += dri_df[age_g] * hc[age_g]
    total = total.rename('Total Dietary Req for Household: ' + str(idx))
    return pd.DataFrame(total)

In [ ]:
# use index of household
idx = household_idx[0]

get_household_char(idx)

In [ ]:
get_dri_household_df(idx, dri_df)

#### Compare if total nutrition consumed meets dietary requirement:

In [ ]:
# example: 7th household 

a = get_total_nutri_household_df(household_idx[6], nutritions)
b = get_dri_household_df(household_idx[6], dri_df)
df = pd.concat([b, a], axis=1)
df['Excess'] = df.iloc[:, 1] - df.iloc[:, 0] 
df

In [ ]:
# check if all values under excess column is more than 0
(df['Excess'].values > 0).all()

#### Function that checks if a particular household meets all DRI requirements:

In [ ]:
# Returns True if all values under 'Excess' column is positive, i.e. all req. met

def check_dri_req(idx, nutritions, dri_df):
    a = get_total_nutri_household_df(idx, nutritions)
    b = get_dri_household_df(idx, dri_df)
    df = pd.concat([b, a], axis=1)
    df['Excess'] = df.iloc[:, 1] - df.iloc[:, 0] 
    return (df['Excess'].values > 0).all()

In [ ]:
# check for 200th household
check_dri_req(household_idx[199], nutritions, dri_df)

#### Run the above function for all households

In [ ]:
# create empty list
households = []

# add indices of households that meet requirements to empty list
for ih in set(household_idx):
    if check_dri_req(idx, nutritions, dri_df):
        households.append(ih)

#### Proportion of all households meeting DRI requirement:

In [ ]:
num_of_h_met_req = len(households)
total_num_h = len(set(household_idx))
proportion = num_of_h_met_req / total_num_h
proportion

#### Proportion of poor households meeting DRI requirement:

In [ ]:
# get indices of all poor households
poor_households_idx = poverty_line_df.set_index(['t', 'j', 'm']).index

# get indices of poor households with consumption data
con_poor_households_idx = (consumption_df[consumption_df.index.isin(poor_households_idx)]).index

In [ ]:
# create empty list
poor_households = []

# add indices of households that meet requirements to empty list
for ip in set(con_poor_households_idx):
    if check_dri_req(idx, nutritions, dri_df):
        poor_households.append(ip)

proportion = len(poor_households) / len(set(con_poor_households_idx))
proportion